In [103]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pathlib


In [104]:
# prepare data_dir
data_dir = "data_dir"
train_dir = pathlib.Path(data_dir + "/train")
test_dir = pathlib.Path(data_dir + "/test")


In [105]:
# get train and test images
train_images = list(train_dir.glob("*/*.*"))
test_images = list(test_dir.glob("*/*.*"))


In [106]:
# prepare training data set
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)


Found 224 files belonging to 2 classes.
Using 180 files for training.


In [107]:
# prepare validation data set
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size)


Found 36 files belonging to 2 classes.
Using 7 files for validation.


In [108]:
# prepare class names
class_names = train_ds.class_names


In [109]:
# Configure the dataset for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [110]:
# normalize data from [0, 255] to [0, 1]
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)


In [111]:
# prepare normalized data set and use it
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))


In [112]:
# set parameters for the loader
img_height = image_batch[0].shape[0]
img_width = image_batch[0].shape[1]
batch_size = image_batch[0].shape[2]


In [113]:
# create model
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(
            1./255, input_shape=(img_height, img_width, batch_size)),
    tf.keras.layers.Flatten(input_shape=(img_height, img_width)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names))
])


In [114]:
# compile model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [115]:
# model summary
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_5 (Rescaling)      (None, 1376, 1038, 3)     0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4284864)           0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               548462720 
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 258       
Total params: 548,462,978
Trainable params: 548,462,978
Non-trainable params: 0
_________________________________________________________________


In [119]:
# train model
epochs = 3
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)


Epoch 1/10
32/60 [===============>..............] - ETA: 48s - loss: 512.3054 - accuracy: 0.7188

KeyboardInterrupt: 

In [117]:
# Evaluate accuracy
test_loss, test_acc = model.evaluate(test_images,  class_names, verbose=2)

print('\nTest accuracy:', test_acc)


ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'pathlib.PosixPath'>"}), (<class 'list'> containing values of types {"<class 'str'>"})